In [45]:
import pandas as pd
import spotipy
from random import sample
from secrets import spotify_secrets, localhost_db, postgres_db
from sqlalchemy import create_engine
from spotipy.oauth2 import SpotifyOAuth
from queries import *

df_a = pd.read_csv('data/TopArtists.csv')
df_t = pd.read_csv('data/TopTracks.csv')
df_g = pd.read_csv('data/TopGenres.csv')
df_m = pd.read_csv('data/MusicFeatures.csv')

In [7]:
u2_a = df_a.loc[df_a['user_id'] == 12120382831]
u2_t = df_t.loc[df_t['user_id'] == 12120382831]
u2_g = df_g.loc[df_g['user_id'] == 12120382831]
u2_m = df_m.loc[df_m['user_id'] == 12120382831]

In [8]:
u2_a.head()

,user_id,rank,artist_id,timeframe,artist,genres,artist_url,artist_image,popularity
1,12120382831,1,5K4W6rqBFWDnAN6FQUkS6x,Long,Kanye West,chicago rap; rap,https://open.spotify.com/artist/5K4W6rqBFWDnAN...,https://i.scdn.co/image/bd1c6fdf3705cf9b7d0c8a...,90
2,12120382831,2,2elBjNSdBE2Y3f0j1mjrql,Long,Jay Chou,c-pop; mandopop; taiwan pop; zhongguo feng,https://open.spotify.com/artist/2elBjNSdBE2Y3f...,https://i.scdn.co/image/0e5952203ea0140aa5687f...,73
4,12120382831,3,73sIBHcqh3Z3NyqHKZ7FOL,Long,Childish Gambino,atl hip hop; hip hop; pop rap; rap,https://open.spotify.com/artist/73sIBHcqh3Z3Ny...,https://i.scdn.co/image/2e1d974c14fb66eb5050bf...,80
7,12120382831,4,1Xyo4u8uXC1ZmMpatF05PJ,Long,The Weeknd,canadian contemporary r&b; canadian pop; pop,https://open.spotify.com/artist/1Xyo4u8uXC1ZmM...,https://i.scdn.co/image/d9a875c37277c35b94c60c...,94
8,12120382831,5,50co4Is1HCEo8bhOyUWKpn,Long,Young Thug,atl hip hop; atl trap; gangster rap; melodic r...,https://open.spotify.com/artist/50co4Is1HCEo8b...,https://i.scdn.co/image/69ef089edd03d6d5455ed6...,90


In [11]:
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=spotify_secrets["Client Id"],
                                               client_secret=spotify_secrets["Client Secret"],
                                               redirect_uri="http://localhost:8892/callback",
                                               scope="user-top-read"))

In [76]:
def recommend_artist(df):
    user_id = df['user_id'].unique()[0]
    recom_list = []
    seed_artists = df['artist_id'].iloc[:5].tolist()
    for seed in seed_artists:
        recom = sp.artist_related_artists(seed)
        for a in recom['artists']:
            if a['id'] not in df['artist_id'].iloc[:20]:
                this_recom = {
                    'user_id': user_id,
                    'artist_id': a['id'],
                    'artist': a['name'],
                    'genres': "; ".join(g for g in a['genres']),
                    'artist_url': a['external_urls']['spotify'],
                    'artist_image': a['images'][0]['url'],
                    'popularity': a['popularity'],
                }
                recom_list.append(this_recom)
    if len(recom_list) >= 20:
        return pd.DataFrame.from_dict(recom_list).sample(n=20)
    else:
        return pd.DataFrame.from_dict(recom_list)

def recommend_track(df_t):
    user_id = df_t['user_id'].unique()[0]
    seed_tracks = df_t['track_id'].iloc[:5].tolist()
    recom = sp.recommendations(seed_tracks=seed_tracks, limit=50)
    recom_list = []
    for t in recom['tracks']:
        if t['id'] not in df_t['track_id'].iloc[:20]:
            this_recom = {
                'user_id': user_id,
                'track_id': t['id'],
                'track_id': t['id'],
                'track': t['name'],
                'artists': "; ".join(a['name'] for a in t['artists']),
                'album': t['album']['name'],
                'album_image': t['album']['images'][0]['url'],
                'release_date': t['album']['release_date'],
                'track_url': t['external_urls']['spotify']
            }
            recom_list.append(this_recom)
        if len(recom_list) >= 30:
            break
    return pd.DataFrame.from_dict(recom_list)

In [71]:
df_ra = recommend_artist(u2_a.loc[u2_a['timeframe'] == 'Short'])
df_ra.head()

,user_id,artist_id,artist,genres,artist_url,artist_image,popularity
64,12120382831,4ehtJnVumNf6xzSCDk8aLB,Martin Jensen,dance pop; danish electronic; edm; electro hou...,https://open.spotify.com/artist/4ehtJnVumNf6xz...,https://i.scdn.co/image/eacedf7ec88b3591afe4e3...,76
38,12120382831,6VbRanWSU3pdDhJnhSfGmY,Harlem Yu,c-pop; classic mandopop; mandopop,https://open.spotify.com/artist/6VbRanWSU3pdDh...,https://i.scdn.co/image/9db6c8bf469228b2f15b25...,50
39,12120382831,3AroL2oDPiAnMpTmIQv3KP,Cyndi Wang,mandopop; taiwan pop,https://open.spotify.com/artist/3AroL2oDPiAnMp...,https://i.scdn.co/image/4d13d5e841cb97a9d3586c...,55
36,12120382831,1Hu58yHg2CXNfDhlPd7Tdd,Jacky Cheung,c-pop; cantopop; classic cantopop; classic man...,https://open.spotify.com/artist/1Hu58yHg2CXNfD...,https://i.scdn.co/image/c90c352bbcffbc57c7930c...,63
69,12120382831,4sTQVOfp9vEMCemLw50sbu,Galantis,big room; dance pop; edm; electro house; pop; ...,https://open.spotify.com/artist/4sTQVOfp9vEMCe...,https://i.scdn.co/image/2df5fb887384d10f117938...,76


In [77]:
df_rt = recommend_track(u2_t.loc[u2_t['timeframe'] == 'Short'])
df_rt.head()

,user_id,track_id,track,artists,album,album_image,release_date,track_url
0,12120382831,7Fk8nem6GNCNbAQ7qjO7mj,The Boys of Summer,Canyon City,3 Hours,https://i.scdn.co/image/ab67616d0000b273e1a85b...,2018-06-15,https://open.spotify.com/track/7Fk8nem6GNCNbAQ...
1,12120382831,7H3nOoaHiPJPKadrHcMiex,Face To Face,Mat Kearney,CRAZYTALK,https://i.scdn.co/image/ab67616d0000b273912a10...,2018-05-04,https://open.spotify.com/track/7H3nOoaHiPJPKad...
2,12120382831,5KBKqxYY263Tr0haAu3fMz,Sometimes,My Bloody Valentine,Loveless,https://i.scdn.co/image/ab67616d0000b2730ede77...,1991-11-04,https://open.spotify.com/track/5KBKqxYY263Tr0h...
3,12120382831,5hc71nKsUgtwQ3z52KEKQk,Somebody Else,The 1975,"I like it when you sleep, for you are so beaut...",https://i.scdn.co/image/ab67616d0000b273d3de03...,2016-02-26,https://open.spotify.com/track/5hc71nKsUgtwQ3z...
4,12120382831,5rmcjZTrE9JR1YrbNZDFNW,She's Mine Pt. 2,J. Cole,4 Your Eyez Only,https://i.scdn.co/image/ab67616d0000b273f4ca75...,2016-12-09,https://open.spotify.com/track/5rmcjZTrE9JR1Yr...


In [81]:
df_ra['user_id'] = df_ra['user_id'].astype(str)
df_rt['user_id'] = df_rt['user_id'].astype(str)

In [82]:
engine = create_engine(localhost_db)
df_ra[['user_id', 'artist_id']].to_sql('RecommendArtists', engine, index=False, if_exists='replace')
df_rt[['user_id', 'track_id']].to_sql('RecommendTracks', engine, index=False, if_exists='replace')
df_ra.drop(columns=['user_id']).to_sql('TempArtists', engine, index=False, if_exists='replace')
df_rt.drop(columns=['user_id']).to_sql('TempTracks', engine, index=False, if_exists='replace')
engine.dispose()

In [83]:
engine = create_engine(postgres_db)
df_ra[['user_id', 'artist_id']].to_sql('RecommendArtists', engine, index=False, if_exists='replace')
df_rt[['user_id', 'track_id']].to_sql('RecommendTracks', engine, index=False, if_exists='replace')
engine.dispose()